In [1]:
!pip install -U transformers datasets peft accelerate -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType


In [3]:
dataset = load_dataset("imdb")
dataset


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [4]:
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")


In [7]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_lin", "v_lin"]
)


model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


In [11]:
training_args = TrainingArguments(
    output_dir="./lora_results",
    do_eval=True,
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to='none',
)


In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(2000)),
    eval_dataset=tokenized_datasets["test"].select(range(500)),
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/tmp/ipython-input-13-2598002916.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [14]:
trainer.train()


Step,Training Loss
500,0.376200


TrainOutput(global_step=500, training_loss=0.37621298217773436, metrics={'train_runtime': 62.1973, 'train_samples_per_second': 64.311, 'train_steps_per_second': 8.039, 'total_flos': 269478813696000.0, 'train_loss': 0.37621298217773436, 'epoch': 2.0})

In [15]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

trainer.compute_metrics = compute_metrics
results = trainer.evaluate()
print(results)


{'eval_loss': 0.2964455485343933, 'eval_accuracy': 0.874, 'eval_f1': 0.0, 'eval_runtime': 3.9409, 'eval_samples_per_second': 126.876, 'eval_steps_per_second': 15.986, 'epoch': 2.0}


In [16]:
model.save_pretrained("./lora_finetuned_imdb")
tokenizer.save_pretrained("./lora_finetuned_imdb")


('./lora_finetuned_imdb/tokenizer_config.json',
 './lora_finetuned_imdb/special_tokens_map.json',
 './lora_finetuned_imdb/vocab.txt',
 './lora_finetuned_imdb/added_tokens.json',
 './lora_finetuned_imdb/tokenizer.json')

In [20]:
model.save_pretrained("./lora_finetuned_imdb")
tokenizer.save_pretrained("./lora_finetuned_imdb")


('./lora_finetuned_imdb/tokenizer_config.json',
 './lora_finetuned_imdb/special_tokens_map.json',
 './lora_finetuned_imdb/vocab.txt',
 './lora_finetuned_imdb/added_tokens.json',
 './lora_finetuned_imdb/tokenizer.json')

In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("./lora_finetuned_imdb")
tokenizer = AutoTokenizer.from_pretrained("./lora_finetuned_imdb")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
def inference_engine(text, model, tokenizer, threshold=0.6):
    device = next(model.parameters()).device

    # Move inputs to model device
    inputs = tokenizer(text, return_tensors="pt").to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1)
        confidence, pred = torch.max(probs, dim=-1)
        pred_label = pred.item()

    if any(neg in text.lower() for neg in ["not", "never", "no"]):
        pred_label = 1 - pred_label

    # Confidence threshold
    if confidence.item() < threshold:
        return "Uncertain"

    return "Positive" if pred_label == 1 else "Negative"


In [24]:
inference_engine("This movie was good.", model, tokenizer)


'Positive'

In [25]:
import time

start = time.time()
print(inference_engine("A sample sentence", model, tokenizer))
print("Time:", time.time() - start)


Negative
Time: 0.046637535095214844


In [32]:
import os
os.environ["WANDB_DISABLED"] = "true"

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import time

MODEL_PATH = "lora_finetuned_imdb"

model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

args = TrainingArguments(
    output_dir="./results",
    report_to="none"
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    data_collator=data_collator
)

pred = trainer.predict(tokenized_datasets["test"].select(range(500)))
metrics = compute_metrics(pred)

start_time = time.time()
_ = inference_engine("I did not like this movie at all.", model, tokenizer)
inference_time = (time.time() - start_time) * 1000
!du -sh {MODEL_PATH}

print("\n === METRICS TABLE ===")
print(f"Model: {MODEL_PATH}")
print(f"Accuracy:  {metrics['accuracy']*100:.2f}%")
print(f"Inference Time: ~{inference_time:.1f} ms per sample")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-32-4134982618.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


3.8M	lora_finetuned_imdb

 === METRICS TABLE ===
Model: lora_finetuned_imdb
Accuracy:  87.40%
Inference Time: ~8.2 ms per sample


In [34]:
# !zip -r lora_finetuned_imdb.zip lora_finetuned_imdb


  adding: lora_finetuned_imdb/ (stored 0%)
  adding: lora_finetuned_imdb/adapter_model.safetensors (deflated 7%)
  adding: lora_finetuned_imdb/adapter_config.json (deflated 56%)
  adding: lora_finetuned_imdb/tokenizer_config.json (deflated 73%)
  adding: lora_finetuned_imdb/tokenizer.json (deflated 71%)
  adding: lora_finetuned_imdb/README.md (deflated 66%)
  adding: lora_finetuned_imdb/vocab.txt (deflated 53%)
  adding: lora_finetuned_imdb/special_tokens_map.json (deflated 80%)
